In [7]:
import asyncio
import ssl
import json
import websockets
from websockets_proxy import Proxy, proxy_connect
import nest_asyncio
import argparse
import asyncio
import random
import signal
import ssl
import json
import time
import uuid
from loguru import logger
from websockets_proxy import Proxy, proxy_connect
import os

In [8]:
async def connect_to_wss(http_proxy, user_id):
    device_id = str(uuid.uuid3(uuid.NAMESPACE_DNS, http_proxy))
    logger.info(device_id)
    while True:
        try:
            await asyncio.sleep(random.randint(1, 10) / 10)
            custom_headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
            }
            ssl_context = ssl.create_default_context()
            ssl_context.check_hostname = False
            ssl_context.verify_mode = ssl.CERT_NONE
            uri = "wss://proxy.wynd.network:4650/"
            server_hostname = "proxy.wynd.network"
            proxy = Proxy.from_url(http_proxy)
            async with proxy_connect(uri, proxy=proxy, ssl=ssl_context, server_hostname=server_hostname,
                                     extra_headers=custom_headers) as websocket:
                async def send_ping():
                    while True:
                        send_message = json.dumps(
                            {"id": str(uuid.uuid4()), "version": "1.0.0", "action": "PING", "data": {}})
                        logger.debug(send_message)
                        await websocket.send(send_message)
                        await asyncio.sleep(20)

                await asyncio.sleep(1)
                asyncio.create_task(send_ping())

                while True:
                    response = await websocket.recv()
                    message = json.loads(response)
                    logger.info(message)
                    if message.get("action") == "AUTH":
                        auth_response = {
                            "id": message["id"],
                            "origin_action": "AUTH",
                            "result": {
                                "browser_id": device_id,
                                "user_id": user_id,
                                "user_agent": custom_headers['User-Agent'],
                                "timestamp": int(time.time()),
                                "device_type": "extension",
                                "version": "2.5.0"
                            }
                        }
                        logger.debug(auth_response)
                        await websocket.send(json.dumps(auth_response))

                    elif message.get("action") == "PONG":
                        pong_response = {"id": message["id"], "origin_action": "PONG"}
                        logger.debug(pong_response)
                        await websocket.send(json.dumps(pong_response))
        except asyncio.CancelledError:
            logger.info(f"Task for proxy {http_proxy} cancelled")
            break
        except Exception as e:
            logger.error(e)
            logger.error(http_proxy)

In [11]:
import asyncio

async def test_connect_to_wss(http_proxy, user_id):
    try:
        await getgrass_proxy.connect_to_wss(http_proxy, user_id)
    except Exception as e:
        print(f"Error occurred while testing proxy {http_proxy}: {e}")

# Example usage
if __name__ == '__main__':
    proxy_to_test = "socks4://81.12.119.171:8080"  # Replace with the proxy you want to test
    user_id = ""  # Replace with your user ID
    
    # Run the coroutine in an event loop
    asyncio.run(test_connect_to_wss(proxy_to_test, user_id))


RuntimeError: Cannot run the event loop while another loop is running

Failed to connect with proxy socks5://45.6.15.119:5678: [Errno 104] Connection reset by peer
Proxy connection success: False
